In [2]:
# Importieren der notwendigen Bibliotheken
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from tqdm import tqdm

# Daten laden
# Die CSV-Datei 'processed_sentiment140.csv' wird geladen. Sie enthält die Textdaten ('processed_text') und ihre zugehörigen Sentimentlabels ('Sentiment').
data = pd.read_csv('processed_sentiment140.csv')  # Passe den Pfad an

# Behandlung von fehlenden Werten
# NaN-Werte in der Spalte 'processed_text' werden durch leere Zeichenfolgen ersetzt, um Probleme bei der Textverarbeitung zu vermeiden.
data['processed_text'] = data['processed_text'].fillna('')

# Aufteilen in Merkmale und Ziel
# 'processed_text' ist die Eingabedaten (X), 'Sentiment' ist das Ziel (y).
X = data['processed_text']
y = data['Sentiment']

# Datenaufteilung in Trainings- und Testdaten
# Die Daten werden in Trainingsdaten (80%) und Testdaten (20%) aufgeteilt, wobei die Verteilung der Labels mit 'stratify=y' gleich bleibt.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# TF-IDF-Transformation
# Konvertiert die Texte in numerische Repräsentationen (Feature-Vektoren) mithilfe von TF-IDF.
# max_features=2000 beschränkt die Anzahl der Features auf die 2000 häufigsten.
# ngram_range=(1, 2) berücksichtigt Unigramme (einzelne Wörter) und Bigramme (Wortpaare).
tfidf_vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)  # Trainingstransformation
X_test_tfidf = tfidf_vectorizer.transform(X_test)  # Testtransformation

# Fortschrittsanzeige für das Training
print("Training des Modells...")
for _ in tqdm(range(1), desc="Training Fortschritt"):
    # Lineares Regressionsmodell
    # Ein lineares Regressionsmodell wird trainiert, um die Sentiment-Scores vorherzusagen.
    linear_model = LinearRegression()
    linear_model.fit(X_train_tfidf, y_train)  # Modelltraining mit den TF-IDF-transformierten Trainingsdaten

# Vorhersage und Umwandlung in binäre Werte
print("\nEvaluierung des Modells...")
y_pred = linear_model.predict(X_test_tfidf)  # Vorhersage der Sentiment-Scores auf den Testdaten
# Die vorhergesagten Scores (kontinuierliche Werte) werden in binäre Labels (0 oder 1) umgewandelt:
# Scores < 0.5 werden zu 0, Scores >= 0.5 werden zu 1.
y_pred_rounded = [0 if pred < 0.5 else 1 for pred in y_pred]

# Evaluierung der Vorhersagen
# Berechnung der Mean Squared Error (MSE) als Regressionsmetrik.
# Berechnung der Genauigkeit (Accuracy), indem die gerundeten Vorhersagen mit den tatsächlichen Labels verglichen werden.
# Der Classification Report gibt Precision, Recall und F1-Score für jede Klasse aus.
mse = mean_squared_error(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred_rounded)
classification_report_text = classification_report(y_test, y_pred_rounded)

# Ausgabe der Evaluierungsergebnisse
print(f"\nMean Squared Error: {mse}")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{classification_report_text}")

# Ergebnisse zusammenführen
# Eine Tabelle wird erstellt, die den ursprünglichen Text, das wahre Label und das vorhergesagte Label für jeden Testdatensatz enthält.
results = pd.DataFrame({
    'text': X_test,
    'true_label': y_test,
    'predicted_label': y_pred_rounded
})

# Beispiel korrekt klassifizierter Einträge
# Es werden 5 Beispiele aus den korrekt klassifizierten Testdatensätzen zufällig ausgewählt.
correct_predictions = results[results['true_label'] == results['predicted_label']].sample(5, random_state=42)

# Beispiel falsch klassifizierter Einträge
# Es werden 5 Beispiele aus den falsch klassifizierten Testdatensätzen zufällig ausgewählt.
incorrect_predictions = results[results['true_label'] != results['predicted_label']].sample(5, random_state=42)

# Anzeigen der Beispiele
print("\nKorrekt klassifizierte Beispiele:")
print(correct_predictions)

print("\nFalsch klassifizierte Beispiele:")
print(incorrect_predictions)


Training des Modells...


Training Fortschritt: 100%|███████████████████████| 1/1 [00:06<00:00,  6.71s/it]



Evaluierung des Modells...

Mean Squared Error: 0.1657199325574001
Accuracy: 0.76233125
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.73      0.75    160000
           1       0.75      0.79      0.77    160000

    accuracy                           0.76    320000
   macro avg       0.76      0.76      0.76    320000
weighted avg       0.76      0.76      0.76    320000


Korrekt klassifizierte Beispiele:
                                                      text  true_label  \
1116794                                          satisfied           1   
4279     today wear ians hoodie im tired allergy acting...           0   
1526062  haha go buy lol c quotlovatoquot ni imong ka t...           1   
589414                                    doesnt want come           0   
382312              aw suck im melbourne ruby rose awesome           0   

         predicted_label  
1116794                1  
4279                   0  
